In [48]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np

from keras.utils import pad_sequences
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [49]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'orders_and_products_by_customer.pkl')

In [50]:
product_data = product_data.sample(n=1000)
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
30865,3855949.0,[3855946],[[3842542]],[[Schnürschuh Rauleder blau]],[[1]],[[97.43]],[[2020-08-19 19:12:46]]
143171,6610827.0,[6610824],[[4768577]],[[Sneaker low Glattleder weiß]],[[1]],[[79.0]],[[2021-08-14 20:21:58]]
13636,2101813.0,[11109745],[[10561150]],[[Sneaker low Rauleder beige]],[[1]],[[120.0]],[[2023-03-21 14:25:56]]
19178,2173305.0,[4442316],[[3843436]],[[Sneaker high Rauleder gelb]],[[1]],[[97.43]],[[2020-10-20 09:29:51]]
82284,5168729.0,[5168727],[[4769616]],[[Sneaker low Glattleder weiß]],[[1]],[[115.0]],[[2021-02-19 14:17:34]]
...,...,...,...,...,...,...,...
143988,6629730.0,[6629725],[[589811]],[[Eleganter Pumps Glattleder blau]],[[1]],[[89.95]],[[2021-08-17 14:04:35]]
2257,1967021.0,"[3983654, 3995117, 7303500, 7965067]","[[3591689, 3847025, 3694434, 3555096], [388467...","[[Eleganter Pump Rauleder braun, Chelsea Boot ...","[[1, 1, 1, 1], [1], [1], [1, 1, 1]]","[[97.43, 97.43, 116.97, 97.43], [112.1], [115....","[[2020-09-03 18:12:56, 2020-09-03 18:12:56, 20..."
137157,6481528.0,[6481524],"[[3695622, 3695623]]","[[PG0460 Sneaker high Rauleder blau, PG0460 Sn...","[[1, 1]]","[[69.0, 69.0]]","[[2021-07-25 17:40:19, 2021-07-25 17:40:19]]"
98882,5554702.0,[5554699],[[4979166]],[[Sneaker low Materialmix Leder/Lederimitat mu...,[[1]],[[130.0]],[[2021-04-07 12:53:12]]


In [51]:
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
30865,3855949.0,[3855946],[[3842542]],[[Schnürschuh Rauleder blau]],[[1]],[[97.43]],[[2020-08-19 19:12:46]]
143171,6610827.0,[6610824],[[4768577]],[[Sneaker low Glattleder weiß]],[[1]],[[79.0]],[[2021-08-14 20:21:58]]
13636,2101813.0,[11109745],[[10561150]],[[Sneaker low Rauleder beige]],[[1]],[[120.0]],[[2023-03-21 14:25:56]]
19178,2173305.0,[4442316],[[3843436]],[[Sneaker high Rauleder gelb]],[[1]],[[97.43]],[[2020-10-20 09:29:51]]
82284,5168729.0,[5168727],[[4769616]],[[Sneaker low Glattleder weiß]],[[1]],[[115.0]],[[2021-02-19 14:17:34]]
...,...,...,...,...,...,...,...
143988,6629730.0,[6629725],[[589811]],[[Eleganter Pumps Glattleder blau]],[[1]],[[89.95]],[[2021-08-17 14:04:35]]
2257,1967021.0,"[3983654, 3995117, 7303500, 7965067]","[[3591689, 3847025, 3694434, 3555096], [388467...","[[Eleganter Pump Rauleder braun, Chelsea Boot ...","[[1, 1, 1, 1], [1], [1], [1, 1, 1]]","[[97.43, 97.43, 116.97, 97.43], [112.1], [115....","[[2020-09-03 18:12:56, 2020-09-03 18:12:56, 20..."
137157,6481528.0,[6481524],"[[3695622, 3695623]]","[[PG0460 Sneaker high Rauleder blau, PG0460 Sn...","[[1, 1]]","[[69.0, 69.0]]","[[2021-07-25 17:40:19, 2021-07-25 17:40:19]]"
98882,5554702.0,[5554699],[[4979166]],[[Sneaker low Materialmix Leder/Lederimitat mu...,[[1]],[[130.0]],[[2021-04-07 12:53:12]]


In [52]:
display(product_data[['customer_id', 'order_ids', 'product_ids', 'amounts', 'order_dates']])

,customer_id,order_ids,product_ids,amounts,order_dates
30865,3855949.0,[3855946],[[3842542]],[[1]],[[2020-08-19 19:12:46]]
143171,6610827.0,[6610824],[[4768577]],[[1]],[[2021-08-14 20:21:58]]
13636,2101813.0,[11109745],[[10561150]],[[1]],[[2023-03-21 14:25:56]]
19178,2173305.0,[4442316],[[3843436]],[[1]],[[2020-10-20 09:29:51]]
82284,5168729.0,[5168727],[[4769616]],[[1]],[[2021-02-19 14:17:34]]
...,...,...,...,...,...
143988,6629730.0,[6629725],[[589811]],[[1]],[[2021-08-17 14:04:35]]
2257,1967021.0,"[3983654, 3995117, 7303500, 7965067]","[[3591689, 3847025, 3694434, 3555096], [388467...","[[1, 1, 1, 1], [1], [1], [1, 1, 1]]","[[2020-09-03 18:12:56, 2020-09-03 18:12:56, 20..."
137157,6481528.0,[6481524],"[[3695622, 3695623]]","[[1, 1]]","[[2021-07-25 17:40:19, 2021-07-25 17:40:19]]"
98882,5554702.0,[5554699],[[4979166]],[[1]],[[2021-04-07 12:53:12]]


In [53]:
# Compute the number of unique order IDs
num_orders = 0
for order_ids in product_data['order_ids']:
    num_orders += len(order_ids)

num_products = 0
for product_ids in product_data['product_ids']:
    num_products += len(product_ids)

num_amounts = 0
for amounts in product_data['amounts']:
    num_amounts += len(amounts)

print(num_orders, num_products, num_amounts)

1290 1290 1290


In [54]:
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from keras.models import Model

# Define the inputs
order_inputs = Input(shape=(None,), dtype='int32', name='order_inputs')
product_inputs = Input(shape=(None,), dtype='int32', name='product_inputs')
amount_inputs = Input(shape=(None,), dtype='int32', name='amount_inputs')

# Define the embedding layers
order_emb = Embedding(input_dim=num_orders, output_dim=16)(order_inputs)
product_emb = Embedding(input_dim=num_products, output_dim=16)(product_inputs)
amount_emb = Embedding(input_dim=num_amounts, output_dim=16)(amount_inputs)

# Concatenate the embeddings
concat_emb = Concatenate()([order_emb, product_emb, amount_emb])

# Define the LSTM layer
lstm = LSTM(units=64)(concat_emb)

# Define the output layer
output = Dense(units=1, activation='linear')(lstm)

# Define the model
model = Model(inputs=[order_inputs, product_inputs, amount_inputs], outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])


In [57]:
from sklearn.model_selection import train_test_split

# Define the input and output data
X = product_data[['order_ids', 'product_ids', 'amounts']]
y = product_data['amounts']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
#history = model.fit(x=X, y=y, batch_size=32, epochs=10)
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
# Evaluate the model
loss, mse = model.evaluate([product_data.order_dates_test, product_data.product_ids_test, product_data.total_prices_test], product_data.total_prices_test)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))

In [ ]:
# split the data into training and testing sets
#X = pd.get_dummies(product_data['product__id'], prefix='product')
#y = product_data['amount']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)